In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"

In [2]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [3]:
def grade_by_model(test_case, output):
    # Create evaluation prompt
    eval_prompt = """
    You are an expert code reviewer. Evaluate this AI-generated solution.
    
    Task: {task}
    Solution: {solution}
    
    Provide your evaluation as a structured JSON object with:
    - "strengths": An array of 1-3 key strengths
    - "weaknesses": An array of 1-3 key areas for improvement  
    - "reasoning": A concise explanation of your assessment
    - "score": A number between 1-10
    """
    
    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [ ]:
# Function to generate a new dataset
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code
* Respond only with Python, JSON, or a plain Regex
* Do not add any comments or commentary or explanation

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [5]:
import json

dataset = generate_dataset()
print(json.dumps(dataset, indent=2))

[
  {
    "task": "Create a regular expression to validate an AWS IAM username, which must be between 1-64 characters, can only contain alphanumeric characters and certain symbols (_+=,.@-), and cannot start with a hyphen",
    "format": "regex"
  },
  {
    "task": "Write a Python function that generates a formatted AWS S3 bucket policy allowing read-only access to a specific bucket for an IAM role",
    "format": "python"
  },
  {
    "task": "Create a JSON object representing an AWS CloudWatch alarm configuration for monitoring high CPU utilization on an EC2 instance",
    "format": "json"
  }
]


In [6]:
with open('dataset.json', 'w') as f:
    json.dump(dataset, f, indent=2)

In [7]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""
    
    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [8]:
# Functions to validate the output structure
import re
import ast


def validate_json(text):
    try:
        json.loads(text.strip())
        return 10
    except json.JSONDecodeError:
        return 0


def validate_python(text):
    try:
        ast.parse(text.strip())
        return 10
    except SyntaxError:
        return 0


def validate_regex(text):
    try:
        re.compile(text.strip())
        return 10
    except re.error:
        return 0


def grade_syntax(response, test_case):
    format = test_case["format"]
    if format == "json":
        return validate_json(response)
    elif format == "python":
        return validate_python(response)
    else:
        return validate_regex(response)


In [9]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)
    
    # TODO - Grading
    # Grade the output
    model_grade = grade_by_model(test_case, output)
    model_score = model_grade["score"]
    syntax_score = grade_syntax(output, test_case)

    score = (model_score + syntax_score) / 2
    
    return {
        "output": output,
        "test_case": test_case,
        "score": score
    }

In [10]:
from statistics import mean

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []
    
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)
    
    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")
    
    return results

In [11]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

print(json.dumps(results, indent=2))

Average score: 1.6666666666666667
[
  {
    "output": "Here's a regular expression that meets the specified requirements for an AWS IAM username:\n\n```\n^[a-zA-Z0-9+=,.@_][a-zA-Z0-9+=,.@_-]{0,63}$\n```\n\nLet's break down the regex:\n\n- `^` - Start of the string\n- `[a-zA-Z0-9+=,.@_]` - First character must be alphanumeric or one of the allowed symbols, but cannot start with a hyphen\n- `[a-zA-Z0-9+=,.@_-]{0,63}` - Followed by 0-63 additional characters that can include alphanumeric characters and the allowed symbols\n- `$` - End of the string\n\nThis regex ensures that:\n- The username is 1-64 characters long\n- Starts with an alphanumeric character or allowed symbol (not a hyphen)\n- Contains only alphanumeric characters and the specified symbols (_+=,.@-)\n\nExamples of valid usernames:\n- john_doe\n- user123\n- admin@company\n- user-name123\n\nExamples of invalid usernames:\n- -username (starts with a hyphen)\n- username! (contains an unallowed symbol)\n- a{very_long_username_tha